In [1]:
from model.LSTM_r import make_model
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
DEVICE = torch.device('cpu')
print(DEVICE)

cpu


In [3]:
model = make_model(DEVICE, 3, 100, 1)

In [4]:
model

LSTM_submodule(
  (lstm): LSTM(3, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [5]:
X_train = torch.rand((20, 30, 3, 12))
y_train = torch.rand((20, 30, 3))

In [6]:
criterion = nn.L1Loss().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
outputs = model(X_train)

In [8]:
outputs.shape

torch.Size([20, 30, 12])